In this notebook we will attempt to fit a continuous power law model with a difference function, $D$.
$$
f(x \mid \boldsymbol\theta, \boldsymbol\phi) = \frac{p(x \mid \boldsymbol\theta) D(x; \boldsymbol\phi)}{\kappa(\boldsymbol\theta, \boldsymbol\phi)}
$$

In particular, we will be using

$$
p(x \mid \alpha, x_{\text{min}}) \propto x^{-\alpha}, \: x > x_{\text{min}}.
$$
and 
$$
 D(x; \boldsymbol\phi) = 1- \exp\left(-\phi_0 - \phi_1(x-1)\right)
$$
which lead to 
$$
\kappa(\boldsymbol\theta, \boldsymbol\phi) =  \int_{ x_{\text{min}}}^\infty p(t \mid \boldsymbol\theta) D(t; \boldsymbol\phi) dt,\\
= \frac{x_{\text{min}}^{1-\alpha}}{\alpha-1} - \exp(-\phi_0 + \phi_1)\int_{ x_{\text{min}}}^\infty t^{-\alpha}\exp\left(-\phi_1 t \right) dt,\\
=\frac{x_{\text{min}}^{1-\alpha}}{\alpha-1} - \exp(-\phi_0 + \phi_1) \Gamma(1-\alpha, x_{\text{min}} \phi_1 )\phi_1^{\alpha-1}  
$$
Hence
$$
\log(\kappa(\boldsymbol\theta, \boldsymbol\phi)) = \text{log_diff_exp}\left[ (1-\alpha)\log(x_{\text{min}})-\log(\alpha-1), -\phi_0 + \phi_1 + \log(\Gamma(1-\alpha, x_{\text{min}} \phi_1 )) +(\alpha-1)\log(\phi_1) \right].
$$



In [1]:
source("../power_law_aux.r")
library(poweRlaw)
library(rstan)
rstan_options(auto_write = TRUE)

Loading required package: ggplot2
Loading required package: StanHeaders
rstan (Version 2.18.2, GitRev: 2e1f913d3ca3)
For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)


Stan program:

In [2]:
writeLines(readLines("../stan/continuous_power_law_diff_linear.stan"))

functions{
    real gamma_inc(real a, real x){
      int n;
      real ap1;
      real ssum;
      if(a>=0) return gamma_q(a, x) * tgamma(a);
      ap1 = a+1;
      //Get floor(-a)
      n = 0;
      while(n < -a){
        n = n + 1;
      }
      //Get summed part
      {
        vector[n] sums;
        for( i in 0:(n-1) ) sums[i+1] = pow(x, i)/tgamma(ap1+i);
        ssum = sum(sums);
      }
      return tgamma(a)*(gamma_q(a + n, x)-pow(x, a)*exp(-x)*ssum);
    }
    real power_law_linear_diff_lpdf(real x, real a, real m, real[] phi){
      real ldens = -a * log(x) + log1m_exp(-phi[1] - phi[2] * (x-1));
      return(ldens);
    }
    /*TODO: implement rng for prior and posterior predictive checks*/
}
data{
  int<lower=0> K; // number of unique values  
  real values[K];
  int<lower=0> frequencies[K]; 
  real<lower=0> x_min;
  real<lower=0> alpha_shape;
  real<lower=0> alpha_rate;
}
parameters{
  real<lower=1, upper=4> alpha;
  real<lower=0> phi[2];
}
transformed parameters{
    real 

In [3]:
contplawLinearDiff <- readRDS("../stan/continuous_power_law_diff_linear.rds")

In [4]:
data("moby")
the.data <- moby
cdata <- compress_data(the.data)
pl.data <- list(
  K = cdata$K,
  values = cdata$v,
  frequencies = cdata$fs,
  x_min = 1,
  alpha_shape = .2,
  alpha_rate = .1
)

In [ ]:
posterior.contplawLinearDiff <- sampling(contplawLinearDiff, data = pl.data)

In [ ]:
posterior.contplawDiff

In [ ]:
stan_trace(posterior.contplawDiff)

As you can see, Stan could not explore the posterior _at all_. Either there is a serious bug or the model has serious pathologies. Both, probably.